# Complex body TFA Inversion

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA) data on a regular grid of a model with a complex geometry.

In [64]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

### Auxiliary functions

In [65]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [66]:
model_dir = 'data/model.pickle'
data_dir = 'data/data.pickle'
#data_dir = 'eqlayer/direction_inc65_dec-40_h190m/eqlayer_depth190m.pickle'

In [67]:
with open(model_dir) as w:
        model = pickle.load(w)

### Observation points and observed data

In [68]:
with open(data_dir) as w:
        data = pickle.load(w)

In [69]:
# observed data and observation points
dobs = data['tfa_upward_obs']
xp = data['grid'][0]
yp = data['grid'][1]
zp = data['grid'][2] - 1000.
N = data['grid'][3]

In [70]:
xp.size

2037

### Parameters of the initial model

In [71]:
M = 30 # number of vertices per prism
L = 10 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization
incs = model['inc']
decs = model['dec']
intensity = model['intensity']

# depth to the top, thickness and radius
z0 = model['z0']
dz = 800.
r = 600.

# total magnetization
props = {'magnetization': utils.ang2vec(
        intensity, incs, decs)}
rin = np.zeros(M) + r
m0 = np.hstack((rin, np.zeros(2)))
m0 = np.resize(m0, P - 1) 
m0 = np.hstack((m0, dz)) # inicial parameters vector
model0 = mfun.param2polyprism(m0, M, L, z0, props) # list of classes of prisms

# main field
inc, dec = [-21.5, -18.7]

# predict data
d0 = polyprism.tf(xp, yp, zp, model0, inc, dec)

### Limits

In [72]:
# limits for parameters in meters
rmin = 100.
rmax = 3000.
x0min = -1500.
x0max = 1500.
y0min = -1500.
y0max = 1500.
dzmin = 200.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [73]:
# variation for derivatives
deltax = 0.01*np.max(x0max)
deltay = 0.01*np.max(y0max)
deltar = 0.01*np.max(rmax)
deltaz = 0.01*np.max(dzmax)

### Outcropping parameters

In [74]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Folder to save the results

In [75]:
if data['grid'][4] == 'flightlines':
    if incs == model['inc'] and decs == model['dec']:
        mypath = data_dir[:-11]+'tfa_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = data_dir[:-11]+'tfa_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
else:
    if incs == model['inc'] and decs == model['dec']:
        mypath = data_dir[:-11]+'tfa_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = data_dir[:-11]+'tfa_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)

In [76]:
# output of inversion
inversion = dict()

### Regularization parameters

In [77]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 1000.00
dlamb = 10.      # step for Marquadt's parameter

a1 = 0.00000005  # adjacent radial distances within each prism
a2 = 0.00000003   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 0.000005     # vertically adjacent origins
a6 = 0.00000000001   # zero order Tikhonov on adjacent radial distances
a7 = 0.0000004     # zero order Tikhonov on thickness of each prism

In [78]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [79]:
itmax = 30
itmax_marq = 10
tol = 1.0e-4     # stop criterion

In [80]:
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs

In [81]:
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
inversion['initial_estimate'] = model0
inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = alpha
inversion['tol'] = tol
inversion['main_field'] = [-21.5, -18.7]

### Inversion

In [82]:
d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
    xp, yp, zp, m0, M, L, delta,
    itmax, itmax_marq, lamb,
    dlamb, tol, mmin, mmax,
    m_out, dobs, inc, dec,
    props, alpha, z0, dz
)

it:  0   it_marq:  0   lambda: 1e+03   misfit: 1.14496e+05
it:  1   it_marq:  0   lambda: 1e+02   misfit: 3.23845e+04
it:  2   it_marq:  0   lambda: 1e+01   misfit: 5.66543e+03
it:  3   it_marq:  0   lambda: 1e+00   misfit: 3.04128e+03
it:  4   it_marq:  0   lambda: 1e-01   misfit: 9.22787e+02
it:  5   it_marq:  0   lambda: 1e-02   misfit: 2.80814e+03
it:  5   it_marq:  1   lambda: 1e-01   misfit: 8.41671e+02
it:  6   it_marq:  0   lambda: 1e-02   misfit: 1.34950e+03
it:  6   it_marq:  1   lambda: 1e-01   misfit: 3.89199e+02
it:  7   it_marq:  0   lambda: 1e-02   misfit: 1.61863e+04
it:  7   it_marq:  1   lambda: 1e-01   misfit: 3.50226e+02
it:  8   it_marq:  0   lambda: 1e-02   misfit: 1.84066e+04
it:  8   it_marq:  1   lambda: 1e-01   misfit: 6.54509e+03
it:  8   it_marq:  2   lambda: 1e+00   misfit: 3.88001e+03
it:  8   it_marq:  3   lambda: 1e+01   misfit: 2.87188e+03
it:  8   it_marq:  4   lambda: 1e+02   misfit: 2.20853e+03
it:  8   it_marq:  5   lambda: 1e+03   misfit: 1.79505e+

# Results

In [83]:
inversion['data_fit'] = d_fit
inversion['estimate'] = m_est
inversion['prisms'] = model_est
inversion['estimated_models'] = model_list
inversion['objective'] = phi_list
inversion['residual'] = dobs - d_fit
inversion['residual_list'] = res_list

In [84]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)

### 